# Uniqueness Studies 

In this notebook, we study the exact reconstruction from noiseless measurements, to understand better how many and what kind of measurements we need to reconstruct a trajectory. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
%reload_ext autoreload
%autoreload 2

np.set_printoptions(precision=2)

## Example Setup

A sample setup and reconstruction result.

In [ ]:
from trajectory import Trajectory
from environment import Environment
from global_variables import DIM

from measurements import create_mask, get_D_topright
from solvers import alternativePseudoInverse, exactSolution

n_anchors = 4 #number of anchors
n_positions = 10 #number of robot sample positions
n_complexity = 3 #model complexity

trajectory = Trajectory(n_complexity, dim=DIM, model='polynomial')
trajectory.set_coeffs(seed=2)
environment = Environment(n_anchors)

basis = trajectory.get_basis(n_samples=n_positions)

sample_points = trajectory.get_sampling_points(basis=basis)

D_complete = get_D_topright(environment.anchors, sample_points)
mask = create_mask(n_positions, n_anchors, strategy='single', 
                   dim=DIM, n_complexity=n_complexity, seed=2)
D_missing = np.multiply(D_complete, mask)

X = exactSolution(D_missing, environment.anchors, basis, 'least_squares', verbose=True)
traj = trajectory.copy()
traj.set_coeffs(coeffs=X)

environment.set_random_anchors(seed=2)
plt.figure()
environment.plot()
trajectory.plot(basis, color='orange', label='ground truth')
traj.plot(basis, color='blue', marker='x', linestyle=':', label='reconstructed')
plt.legend()

## Simulations

Looks like everything is working. Now let's try many random setups and reconstructions. 

A few first observations from cell below: 

* It looks like when we use exactly K*D measurements, such as with strategy 'simple', there is almost never a unique solution. To test, set n_positions to anything above 4, and use strategy 'simple'.
* When we have 2 * n_complexity + 1 measurements, even if at each point we only see one anchor, there is always a unique solution. To see this, use strategy 'single' and set n_positions to 2 * n_complexity + 1. If we only remove one measurement (n_positions = 2 * n_complexity) this starts breaking. 


In [ ]:
n_it = 100

# Problem: this actually returns garbage if the first estimate is not feasible. 
# See here: https://github.com/scipy/scipy/issues/7618
#method = 'minimize'  

# Problem: this returns the least squares estimate, but does not impose the quadratic constraints. 
# the residuals of this might be non-zero.
method = 'least_squares'

# Problem: produces memory error even for small problem size. 
#method = 'grid'

# Problem: can only take into account exactly K*dim constraints. 
#method = 'roots'


# the first point sees dim+1 anchors, the next K-2 points see the first two anchors, the Kth point sees only one. 
#strategy = 'simple'

# each point sees exactly one anchor. at least d+1 anchors are seen.
strategy = 'single'

# uniformly delete a few 
#strategy = 'uniform'

n_positions = 2 * n_complexity

for i in range(n_it):
    # create random trajectory
    trajectory.set_coeffs(seed=i)
    basis = trajectory.get_basis(n_samples=n_positions)
    sample_points = trajectory.get_sampling_points(basis=basis)

    D_complete = get_D_topright(environment.anchors, sample_points)
    mask = create_mask(n_positions, n_anchors, strategy=strategy, dim=DIM, n_complexity=n_complexity, seed=i)
    
    # trying to get more unique solutions by adding 
    # mask[-1, -1] = 1.0
    
    D_missing = np.multiply(D_complete, mask)

    try:
        X = exactSolution(D_missing, environment.anchors, basis, method, verbose=True)
        assert np.allclose(X, trajectory.coeffs, atol=1e-3)
        
        #X_noiseless = exactSolution(D_complete, environment.anchors, basis, method)
        #assert np.allclose(X_noiseless, trajectory.coeffs)
        print('Seed {} ok.'.format(i))
        
    except ValueError as e:
        print('ValueError for seed {}:{}'.format(i, e))
        
    except AssertionError as e:
        print('Result not exact for seed:', X, trajectory.coeffs)
        
        # We found an example of inexact reconstruction. 
        # We can thus stop here and continue plotting and 
        # investigating it. 
        break
        
    except Exception as e:
        print('Singular matrix for seed {}? Error message:'.format(i)) 
        raise e

if method == 'roots':
    from exact_solution import objective_root
    print('objective_root found:')
    print(objective_root(X, environment.anchors, basis, D_missing))
    print('objective_root original:')
    print(objective_root(trajectory.coeffs, environment.anchors, basis, D_missing))

In [ ]:
traj_second_solution = trajectory.copy()
traj_second_solution.set_coeffs(coeffs=X)

plt.figure()
plt.title('plot of two ambiguous solutions')

trajectory.plot(basis, mask=mask, color='green')
trajectory.plot_connections(basis, environment.anchors, mask, color='k', linestyle=':', linewidth=0.5)
trajectory.plot_number_measurements(basis, mask, legend=True)

traj_second_solution.plot(basis, mask=mask, color='red')
traj_second_solution.plot_connections(basis, environment.anchors, mask, color='k', linestyle=':', linewidth=0.5)
traj_second_solution.plot_number_measurements(basis, mask, legend=False)
environment.plot()
environment.annotate()

plt.axis('equal')
#plt.xlim([0, 6])
#plt.ylim([0, 6])

# Systematic study

In [ ]:
def get_mask(n_anchors, n_complexity, n_samples, 
             dim, init_method=0, fill_method='', **kwargs):
    # get dim+1 different anchor indicies
    anchors_seen = np.random.choice(range(n_anchors), size=(dim+1), replace=False)
    
    # Assumption: 1., 2., 3. do not matter. 
    # 1. all seen from one point
    points_seen = None
    if init_method == 1: 
        idx = 0
        points_seen = [idx] * len(anchors_seen)
    # 2. all seen from d+1 different points
    elif init_method == 2: 
        points_seen = np.random.choice(range(n_samples), size=len(anchors_seen), replace=False)
    # 3. all seen from random points. 
    elif init_method == 3: 
        for i in range(100):
            test = np.random.choice(range(n_samples), size=len(anchors_seen), replace=True)
    
            if len(test) > len(set(test)):
                points_seen = test
                break
    else:
        points_seen = ()
        anchors_seen = ()
        #print('not initalizing mask')
        
    if points_seen is None:
         raise ValueError('no valid mask found.')
        
    mask = np.zeros((n_samples, n_anchors))
    mask[points_seen, anchors_seen] = 1.0
    
    anchors_not_seen = list(set(range(n_anchors)) -  set(anchors_seen))
    points_not_seen = list(set(range(n_samples)) - set(points_seen))

    # each position sees the same anchor(s)
    if 'A' in fill_method: 
        n_more_anchors = 1
        
        # A.a this anchor is part of previously seen
        if fill_method == 'Aa':
            new_anchors = np.random.choice(anchors_seen, size=n_more_anchors, replace=False)
    
        # A.b this anchor is one that was not seen yet
        elif fill_method == 'Ab':
            new_anchors = np.random.choice(anchors_not_seen, size=n_more_anchors, replace=False)
        
        # TODO something like this is more correct. 
        #points_left = np.random.choice(points_not_seen, size=n_samples-len(points_seen), replace=False) 
        #new_anchors = [new_anchors] * len(points_left) 
        #mask[points_left, new_anchors] = 1.0
        
        new_anchors = [new_anchors] * len(points_not_seen) 
        mask[points_not_seen, new_anchors] = 1.0
    
    # B. We have in total n_positions measurements, but each point sees 
    # exactly d anchors. 
    elif fill_method == 'B': 
        import math
        
        d = kwargs.get('d', 1)
        num_full = math.floor(len(points_not_seen) / d)
        residual = len(points_not_seen) % d
        #print('filling {} with d anchors, and one with {}'.format(num_full, residual))

        num_all = num_full+1 if residual > 0 else num_full

        assert num_full * d + residual == len(points_not_seen)

        # fill full ones
        new_points = np.random.choice(points_not_seen, size=num_all, replace=False)
        new_points = [[p]*d for p in new_points]
        anchors = [np.random.choice(range(n_anchors), size=d, replace=False) for _ in range(num_full)]
        
        mask[new_points[:num_full], anchors] = 1.0

        # fill residual.
        anchor = np.random.choice(range(n_anchors), size=residual, replace=False)
        mask[new_points[-1][:residual], anchor] = 1.0
        
    else:
        pass
        #print('not filling mask.')

    return mask

In [ ]:
dim = trajectory.dim
n_anchors = environment.n_anchors
n_complexity = trajectory.n_complexity
n_samples = 10
fig, axs = plt.subplots(1, 3)
axs[0].matshow(get_mask(n_anchors, n_complexity, n_samples, dim, 1))
axs[1].matshow(get_mask(n_anchors, n_complexity, n_samples, dim, 2))
axs[2].matshow(get_mask(n_anchors, n_complexity, n_samples, dim, 3))

In [ ]:
# not sure if the following make any difference. 
maskAa = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=1, fill_method='Aa')
maskAb = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=1, fill_method='Ab')
fig, axs = plt.subplots(1, 2)
axs[0].matshow(maskAa)
axs[1].matshow(maskAb)

In [ ]:
maskB1 = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=1, fill_method='B', d=1)
maskB2 = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=1, fill_method='B', d=2)
maskB3 = get_mask(n_anchors, n_complexity, n_samples, dim, init_method=0, fill_method='B', d=3)
# show result.
fig, axs = plt.subplots(1, 3)
axs[0].matshow(maskB1)
axs[1].matshow(maskB2)
axs[2].matshow(maskB3)

In [ ]:
# Experiment 1
# always rank 3
init_method = 2  # d+1 different points see d+1 different anchors. 
fill_method = 'Aa' # rest sees one of above anchors.
n_samples = n_complexity * dim + 1 # zero fail rate
n_samples = n_complexity * dim  # 100 % fail rate

# Experiment 2
# always rank 4
init_method = 2  # d+1 different points see d+1 different anchors
fill_method = 'Ab' # rest sees unseen anchor. 
n_samples = n_complexity * dim + 1 # zero fail rate
n_samples = n_complexity * dim  # 70% fail rate

# Experiment 3
# results in rank 2, and 100 % fails. Same for Aa, Ab. 
init_method = 1  # one points sees d+1 different anchors
fill_method = 'Aa' # rest sees one unseen anchor. 
n_samples = n_complexity * (dim + 1) # 100 % fail rate
n_samples = n_complexity * dim + 1 # 100 % fail rate

# Experiment 4 
# always yields rank 3!
init_method = 3  # random anchors, at most duplicate 
fill_method = 'Ab' # rest sees one unseen anchors. 
n_samples = n_complexity * dim + 1 # zero fail rate
n_samples = n_complexity * dim # zero fail rate
n_samples = n_complexity * dim - 1 # 85 % fail rate

# Experiment 5
# sometimes yields rank 2, sometiems 3. 
#init_method = 3  # random anchors, at most duplicate 
#fill_method = 'Aa' # rest sees one unseen anchors. 
#n_samples = n_complexity * dim + 1 # 20 % fail rate, whenever rank 2!
#n_samples = n_complexity * dim # 40 % fail rate, whenever rank 2!
#n_samples = n_complexity * dim - 1 # 85 % fail rate, for rank 3 and rank 2

This raises some important questions....

* If we have n_complexity * dim measurements, Experiment 2 almost always fails while Experiment 4 never does. The only difference is that we force one anchor to have two measurements in Experiment 2. What does that mean? 

* In the above experiments, it looks like the rank of the mask matrix has to be 3 (d+1?) or more for it to work. 

In [ ]:
# try B method with different numbers...

# Experiment 1
init_method = 0  # random anchors, at most duplicate 
fill_method = 'B' # rest sees one unseen anchors. 
d = 1
n_samples = n_complexity * dim + 1 # zero fail rate
n_samples = n_complexity * dim # zero fail rate
n_samples = n_complexity * dim - 1 # 100 % fail rate, not many solutions found. 

# Experiment 2
init_method = 0  # random anchors, at most duplicate 
fill_method = 'B' # rest sees one unseen anchors. 
d = 2
#n_samples = n_complexity * dim + 1 # zero fail rate
#n_samples = n_complexity * dim # 60 % fail rate for rank 2 and 3
n_samples = n_complexity * dim - 1 # 100 % fail rate, not many solutions found. 

# Experiment 3
init_method = 0  # random anchors, at most duplicate 
fill_method = 'B' # rest sees one unseen anchors. 
d = 3
n_samples = n_complexity * 3 + 1 # zero fail rate
n_samples = n_complexity * 3 # zero fail rate
n_samples = n_complexity * 3 - 1 # 35% fail rate, for rank 2 and 3. Sometimes rank 2 passes. 
#n_samples = n_complexity * dim + 1 # 100 % fail rate

In [ ]:
from measurements import get_measurements
method = 'least_squares'
basis, D_complete = get_measurements(trajectory, environment, n_samples=n_samples)

n_it = 20
n_fails = 0
n_skips = 0

for i in range(n_it):
    mask = get_mask(n_anchors, n_complexity, n_samples, dim, 
                    init_method, fill_method, d=d) 
    print('rank', np.linalg.matrix_rank(mask))
    D_missing = D_complete * mask
    try:
        X_list = exactSolution(D_missing, environment.anchors, basis, method, verbose=True)
        X = X_list[0]
    except ValueError as e:
        print(e)
        n_skips += 1
        continue

    if len(X_list) > 1 or np.sum(np.abs(X - trajectory.coeffs)) > 1e-3:
        print('fail:', i)
        n_fails += 1
        
print('fail rate: {} %'.format(n_fails / (n_it - n_skips) * 100))

In [ ]:
plt.matshow(mask)

In [ ]:
print(n_complexity)